In [1]:
import os
import csv
import requests
import bs4
import pandas
import re
import time
import datetime
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
import numpy as np
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import multiprocessing

In [2]:
now = datetime.datetime.now()
year = now.isoformat()[0:4]
month = now.isoformat()[5:7]
day = now.isoformat()[8:10]
date = year+month+day

In [3]:
now.isoformat()[0:10]

'2023-05-24'

In [4]:
url = f'https://finance.naver.com/research/company_list.naver\
?keyword=&brokerCode=&searchType=\
writeDate&writeFromDate=2018-01-01&writeToDate={now.isoformat()[0:10]}&itemName=&itemCode=&x=0&y=0&page='
print(url)

https://finance.naver.com/research/company_list.naver?keyword=&brokerCode=&searchType=writeDate&writeFromDate=2018-01-01&writeToDate=2023-05-24&itemName=&itemCode=&x=0&y=0&page=


In [5]:
page = 1
next_link_text = "다음"
stock_name_list = []
link_list = []

page_url = url + str(page)
response = requests.get(page_url, headers={'User-Agent': 'Mozill4a/5.0'})
soup = bs4.BeautifulSoup(response.text, 'html.parser')
last_page = soup.select('table tr td a')[-6]['href'].split('/')[-1].split('=')[-1]

for _ in tqdm(range(int(last_page))):
    
    page_url = url + str(page)
    response = requests.get(page_url, headers={'User-Agent': 'Mozill4a/5.0'})
    soup = bs4.BeautifulSoup(response.text, 'html.parser')
    stock_name = soup.select('table a.stock_item')[0:30]
    link = soup.select('table td:nth-child(2) a')[:-6]
    for i in range(len(soup.select('table a[target]'))):
        stock_name_list.append(stock_name[i].text)
        link_list.append('https://finance.naver.com/research/' + link[i]['href'])
    page += 1

100%|████████████████████████████████████████████████████████████████████████████████| 722/722 [03:31<00:00,  3.41it/s]


In [6]:
len(link_list)

21650

In [9]:
stock_name_list = []
title_list = []
content_list = []
target_price_list = []
opinion_list = []
broker_list = []
time_list = []
view_cnt_list = []
pdf_link_list = []


for page_url in tqdm(link_list):

    response = requests.get(page_url, headers={'User-Agent': 'Mozill4a/5.0'})
    soup = bs4.BeautifulSoup(response.text, 'html.parser')

    # 종목명
    stock_name = soup.select('th.view_sbj span em')[0].text
    stock_name_list.append(stock_name)

    # 제목
    title = soup.select('th.view_sbj')[0].text.replace('\t', '').strip().split('\n')[1]
    title_list.append(title)

    # 증권사
    broker = soup.select('th.view_sbj')[0].text.replace('\t', '').strip().split('\n')[2].split('|')[0]
    broker_list.append(broker)

    # 업로드 시간
    time = soup.select('th.view_sbj')[0].text.replace('\t', '').strip().split('\n')[2].split('|')[1]
    time_list.append(time)

    # 조회 수
    view_cnt = ''.join(filter(str.isdigit, soup.select('th.view_sbj')[0].text.replace('\t', '').strip().split('\n')[2].split('|')[2]))
    view_cnt_list.append(view_cnt)
    
    # 내용
    content = ':'.join([i.text for i in soup.select('table div font')])
    content_list.append(content)  
    try:
        # 목표가
        target_price = soup.select('em.money > strong')[0].text.replace(',', '')
        target_price_list.append(target_price)
        
        # 의견
        opinion = soup.select('em.coment')[0].text
        opinion_list.append(opinion)
    except:
        target_price_list.append(-999999)
        opinion_list.append('no_opinion')

    try:
        # 리포트 링크
        pdf_link = soup.select('table td div a')[0]['href']
        pdf_link_list.append(pdf_link)
    except:
        pdf_link_list.append('no_pdf')

100%|██████████████████████████████████████████████████████████████████████████| 21650/21650 [1:08:26<00:00,  5.27it/s]


In [10]:
df = pd.DataFrame({'종목명' : stock_name_list,
                   '제목' : title_list,
                   '내용' : content_list,
                   '증권사' : broker_list,
                   '날짜' : time_list,
                   '조회수' : view_cnt_list,
                   '목표가' : target_price_list,
                   '의견' : opinion_list,
                   '리포트링크' : pdf_link_list})

In [37]:
df['fname'] = df['종목명'] + '_' + df['증권사'] + '_' + df['날짜']

In [6]:
df = pd.read_csv('증권사 리포트.csv', index_col = 0)
df['fname'] = df['종목명'] + '_' + df['증권사'] + '_' + df['날짜']
df

,종목명,제목,내용,증권사,날짜,조회수,목표가,의견,리포트링크,fname
0,효성첨단소재,"탄소섬유, 성장 가속화",2024년 1.4만톤으로 1년 앞당겨진 탄소섬유 증설 계획::효성첨단소재가 탄소섬유...,하나증권,2023.05.22,1112,560000,Buy,https://ssl.pstatic.net/imgstock/upload/resear...,효성첨단소재_하나증권_2023.05.22
1,대동,K-농기계 대표주자,오랜 업력 지닌 농기계 전문기업::- 1947년에 설립된 농기계 전문 업체로 오랜 ...,DS투자증권,2023.05.22,671,17000,매수,https://ssl.pstatic.net/imgstock/upload/resear...,대동_DS투자증권_2023.05.22
2,디케이락,"실적 좋고, 모멘텀 있고","투자의견 매수, 목표주가 15,500 원 유지::23F매출액: 1,304억원, 영업...",대신증권,2023.05.22,599,15500,Buy,https://ssl.pstatic.net/imgstock/upload/resear...,디케이락_대신증권_2023.05.22
3,아스플로,국산화는 진행 중,"투자의견 Market Perform, 목표주가 14,500원 유지::23E 매출액:...",대신증권,2023.05.22,761,14500,MarketPerform,https://ssl.pstatic.net/imgstock/upload/resear...,아스플로_대신증권_2023.05.22
4,하이록코리아,해양플랜트는 계속된다,"투자의견 매수, 목표주가 35,000 원 유지::23F 매출액: 2,180억원, 영...",대신증권,2023.05.22,440,35000,Buy,https://ssl.pstatic.net/imgstock/upload/resear...,하이록코리아_대신증권_2023.05.22
...,...,...,...,...,...,...,...,...,...,...
21645,LG전자,"4분기 예상 매출액 14조 4,676억원, 영업이익 2,524억원(LG이노텍 제외)","4분기 예상 매출액 14조 4,676억원, 영업이익 2,524억원(LG이노텍 ::L...",IBK투자증권,2018.01.03,3403,130000,매수,http://imgstock.naver.com/upload/research/comp...,LG전자_IBK투자증권_2018.01.03
21646,코오롱인더,본인 좋아져. 친구가 도와줘. 그래서 Top-Pick,NaN,교보증권,2018.01.02,5029,115000,Buy,http://imgstock.naver.com/upload/research/comp...,코오롱인더_교보증권_2018.01.02
21647,강원랜드,이렇게까지 규제할 필요가 있을까,NaN,하나증권,2018.01.02,2741,37000,Buy,http://imgstock.naver.com/upload/research/comp...,강원랜드_하나증권_2018.01.02
21648,SK하이닉스,호실적은 2018년에도 지속 가능,NaN,하나증권,2018.01.02,9621,110000,Buy,http://imgstock.naver.com/upload/research/comp...,SK하이닉스_하나증권_2018.01.02
